<a href="https://colab.research.google.com/github/brianellis1997/Sarcasm_Detection/blob/Lindsey/DS_440_Traditional_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sarcasm Detection using NLP Techniques
### Team 2: Brian Ellis, Lindsey Rich, Elena Kern

## Load Libraries and Dataset

In [1]:
import pandas as pd
import torch
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#test comment

In [2]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [3]:
!git clone https://github.com/brianellis1997/Sarcasm_Detection.git

Cloning into 'Sarcasm_Detection'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (110/110), done.
Receiving objects: 100% (110/110), 3.36 MiB | 8.23 MiB/s, done.
remote: Total 110 (delta 57), reused 3 (delta 0), pack-reused 0
Resolving deltas: 100% (57/57), done.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
train_bal = pd.read_csv('/content/drive/MyDrive/Sarcasm_Data/Train_Balanced.csv')   # Make sure path is correct in your google drive
train_bal.head()

,Unnamed: 0,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,522142,0,"I personally wasn't a huge Garrosh fan, I've a...",cromemako83,AskReddit,2,2,0,2015-07-01,2015-07-11 01:55:53,Fuck Vol'jin. Garrosh Hellscream did nothing w...
1,907864,1,you forgot the,_SharkWeek_,AskReddit,1,1,0,2013-03-01,2013-03-14 03:03:46,That's a lie fed to you by the LIEberal media....
2,604170,1,"Nah man, she's clearly an ad carry",jdswift13,leagueoflegends,1,1,0,2015-10-01,2015-10-21 23:22:17,she isnt already?
3,110635,1,This sub in a nutshell.,trickz-M-,GlobalOffensive,1,-1,-1,2016-12-01,2016-12-05 03:50:18,Cloud 9 Qualify! (ONLY C9 FANS ALLOWED(
4,997758,0,Yes... I do.,guriboysf,videos,4,4,0,2010-01-01,2010-01-17 21:32:40,"""so, i hear you have a fat cock."""


In [6]:
test = pd.read_csv('/content/drive/MyDrive/Sarcasm_Data/Test_Balanced.csv')

In [7]:
train_bal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 808618 entries, 0 to 808617
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Unnamed: 0      808618 non-null  int64 
 1   label           808618 non-null  int64 
 2   comment         808618 non-null  object
 3   author          808618 non-null  object
 4   subreddit       808618 non-null  object
 5   score           808618 non-null  int64 
 6   ups             808618 non-null  int64 
 7   downs           808618 non-null  int64 
 8   date            808618 non-null  object
 9   created_utc     808618 non-null  object
 10  parent_comment  808618 non-null  object
dtypes: int64(5), object(6)
memory usage: 67.9+ MB


In [8]:
train_bal['label'].value_counts()

1    404369
0    404249
Name: label, dtype: int64

We can see our dataset is balanced.

In [9]:
train_bal['label'].value_counts(normalize=True)

1    0.500074
0    0.499926
Name: label, dtype: float64

## EDA

In [10]:
# Separating sarcastic vs non sarcastic comments
sarcastic_com = train_bal[train_bal['label'] == 1]
non_sarcastic_com = train_bal[train_bal['label'] == 0]

In [11]:
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
nltk.download('punkt')
nltk.download('stopwords')

import string

import gensim
from gensim.models.phrases import Phraser, Phrases
from gensim.models.word2vec import Word2Vec

import spacy # for a lemmatization example

from sklearn.manifold import TSNE

import pandas as pd
from bokeh.io import output_notebook, output_file
from bokeh.plotting import show, figure

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
# import nltk
# from nltk.corpus import stopwords

# # Download stopwords if not already downloaded
# nltk.download('stopwords')

# # Get the English stopwords
# stpwrds = set(stopwords.words('english'))

# # Assuming 'comment' is the column containing the sentences in your DataFrame
# comments = train_bal['comment']

# # Function to tokenize, lowercase, and remove stopwords from each comment
# def clean_comments(comments):
#     clean_comments_list = []
#     for comment in comments:
#         # Tokenize and lowercase
#         tokens = comment.lower().split()

#         # Remove stopwords
#         cleaned_tokens = [token for token in tokens if token not in stpwrds]

#         clean_comments_list.append(cleaned_tokens)

#     return clean_comments_list

In [13]:
# sarcastic_comments = sarcastic_com['comment']
# non_sarcastic_comments = non_sarcastic_com['comment']

# # Apply the function to sarcastic and non-sarcastic comments
# sarcastic_clean = clean_comments(sarcastic_comments)
# non_sarcastic_clean = clean_comments(non_sarcastic_comments)

# # Example output for the first 10 sarcastic comments
# print(sarcastic_clean[:10])

In [14]:
# from gensim.models.phrases import Phraser, Phrases

# # Add common two-word phrases aka Bi-grams
# sarc_bigrams = Phraser(Phrases(sarcastic_clean, min_count=32, threshold=64))
# non_sarc_bigrams = Phraser(Phrases(non_sarcastic_clean, min_count=32, threshold=64))

# # Apply the bigram models to the tokenized lists
# clean_sarc = [sarc_bigrams[sentence] for sentence in sarcastic_clean]
# clean_non_sarc = [non_sarc_bigrams[sentence] for sentence in non_sarcastic_clean]

# # Example output for the first 10 sarcastic comments after adding bigrams
# print(clean_sarc[:10])

In [15]:
# # WordCloud for sarcastic comments
# wordcloud_sarc = WordCloud(width=800, height=400, background_color='white').generate(' '.join([' '.join(sentence) for sentence in clean_sarc]))

# # Display the generated word cloud using matplotlib
# plt.figure(figsize=(10, 5))
# plt.imshow(wordcloud_sarc, interpolation='bilinear')
# plt.axis('off')
# plt.title('Word Cloud for Sarcastic Comments')
# plt.show()


In [16]:
# # WordCloud for sarcastic comments
# wordcloud_sarc = WordCloud(width=800, height=400, background_color='white').generate(' '.join([' '.join(sentence) for sentence in clean_non_sarc]))

# # Display the generated word cloud using matplotlib
# plt.figure(figsize=(10, 5))
# plt.imshow(wordcloud_sarc, interpolation='bilinear')
# plt.axis('off')
# plt.title('Word Cloud for Non-Sarcastic Comments')
# plt.show()

# Visualizing Sarcastic Comments by Subreddit

In [17]:
# subreddit_count = sarcastic_com['subreddit'].value_counts()

In [18]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# top = subreddit_count.head(20)
# plt.figure(figsize=(10, 6))
# sns.barplot(x=top.values, y=top.index)
# plt.xlabel('Number of Sarcastic Comments')
# plt.ylabel('Subreddit')
# plt.title('Frequency of Sarcastic Comments by Subreddit')
# plt.show()


## Train Logistic Regression model (baseline)

In [19]:
from sklearn.model_selection import train_test_split
comments = train_bal['comment']
# Create a new column 'merged_comment' by concatenating 'comment' and 'parent_comment'
train_bal['merged_comment'] = train_bal['comment'] + ' ' + train_bal['parent_comment']

labels = train_bal['label']

X_train, X_test, y_train, y_test = train_test_split(train_bal['merged_comment'], labels, test_size=0.3, random_state = 42)

print(X_train.shape)
print(X_test.shape)

(566032,)
(242586,)


In [20]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import Pipeline

# # build bigrams
# # and minimal word frequency
# tf_idf = TfidfVectorizer(ngram_range=(1, 2), min_df=2, lowercase=False)
# # multinomial logistic regression a.k.a softmax classifier
# logit = LogisticRegression(C=1, n_jobs=-1, solver='lbfgs',
#                            random_state=42, verbose=1)
# # sklearn's pipeline
# tfidf_logit_pipeline = Pipeline([('tf_idf', tf_idf),
#                                  ('logit', logit)])

In [21]:
# %%time
# tfidf_logit_pipeline.fit(X_train, y_train)

In [22]:
# %%time
# valid_pred = tfidf_logit_pipeline.predict(X_test)

In [23]:
# from sklearn.metrics import accuracy_score

# accuracy_score(y_test, valid_pred)

## Stacking with parent comment

In [24]:
# from scipy.sparse import hstack
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import Pipeline
# tf_idf_comment = TfidfVectorizer(ngram_range=(1, 2), min_df=2, lowercase=False)
# tf_idf_parent_comment = TfidfVectorizer(ngram_range=(1, 2), min_df=2, lowercase=False)

# comment_vectors = tf_idf_comment.fit_transform(train_bal['comment'])
# parent_comment_vectors = tf_idf_parent_comment.fit_transform(train_bal['parent_comment'])

# # Concatenate the TF-IDF vectors horizontally
# combined_vectors = hstack([comment_vectors, parent_comment_vectors])

In [25]:
# from scipy.sparse import save_npz
# save_npz('/content/drive/MyDrive/Sarcasm_Data/combined_vectors.npz', combined_vectors)

In [26]:
from scipy.sparse import load_npz
combined_vectors = load_npz('/content/drive/MyDrive/Sarcasm_Data/combined_vectors.npz')

In [27]:
combined_vectors

<808618x1878463 sparse matrix of type '<class 'numpy.float64'>'
	with 42748996 stored elements in Compressed Sparse Row format>

In [28]:
X_train, X_hold_out, y_train, y_hold_out = train_test_split(
    combined_vectors, labels, test_size=0.3, random_state=22
)


X_test, X_eval, y_test, y_eval = train_test_split(
    X_hold_out, y_hold_out, test_size=0.5, random_state=22
)

In [29]:


# logit = LogisticRegression(C=1, n_jobs=-1, solver='lbfgs',
#                            random_state=42, verbose=1)

# logit.fit(X_train, y_train)
# valid_pred = logit.predict(X_test)

# from sklearn.metrics import accuracy_score

# accuracy_score(y_test, valid_pred)

## Try Xgboost

In [30]:
train_bal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 808618 entries, 0 to 808617
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Unnamed: 0      808618 non-null  int64 
 1   label           808618 non-null  int64 
 2   comment         808618 non-null  object
 3   author          808618 non-null  object
 4   subreddit       808618 non-null  object
 5   score           808618 non-null  int64 
 6   ups             808618 non-null  int64 
 7   downs           808618 non-null  int64 
 8   date            808618 non-null  object
 9   created_utc     808618 non-null  object
 10  parent_comment  808618 non-null  object
 11  merged_comment  808618 non-null  object
dtypes: int64(5), object(7)
memory usage: 74.0+ MB


In [31]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
deval = xgb.DMatrix(X_eval, label=y_test)

In [50]:
params = {
    'tree_method': 'hist',
    'device': 'cuda',
    'predictor': 'gpu_predictor',
    'max_depth': 4,
    'n_estimators': 100,
    'eta': 0.01,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'n_jobs': -1,
    'lambda': 1,
    'alpha': 0.01

}
num_boost_round = 10000

In [51]:
bst = xgb.train(
    params,
    dtrain,
    num_boost_round,
    evals=[(dtrain, 'train'), (dtest, 'test')],
    early_stopping_rounds=50
)


[0]	train-logloss:0.69273	test-logloss:0.69272


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [17:03:30] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators", "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Streaming output truncated to the last 5000 lines.
[5000]	train-logloss:0.59266	test-logloss:0.59611
[5001]	train-logloss:0.59266	test-logloss:0.59611
[5002]	train-logloss:0.59265	test-logloss:0.59610
[5003]	train-logloss:0.59265	test-logloss:0.59610
[5004]	train-logloss:0.59264	test-logloss:0.59610
[5005]	train-logloss:0.59264	test-logloss:0.59609
[5006]	train-logloss:0.59263	test-logloss:0.59609
[5007]	train-logloss:0.59263	test-logloss:0.59608
[5008]	train-logloss:0.59262	test-logloss:0.59608
[5009]	train-logloss:0.59261	test-logloss:0.59608
[5010]	train-logloss:0.59261	test-logloss:0.59607
[5011]	train-logloss:0.59260	test-logloss:0.59607
[5012]	train-logloss:0.59260	test-logloss:0.59606
[5013]	train-logloss:0.59259	test-logloss:0.59606
[5014]	train-logloss:0.59259	test-logloss:0.59605
[5015]	train-logloss:0.59258	test-logloss:0.59604
[5016]	train-logloss:0.59257	test-logloss:0.59604
[5017]	train-logloss:0.59257	test-logloss:0.59603
[5018]	train-logloss:0.59256	test-logloss:0.59603

In [54]:
bst.save_model('/content/drive/MyDrive/Sarcasm_Data/Xgboost1.model')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [17:26:21] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [55]:
loaded_bst = xgb.Booster()
loaded_bst.load_model('/content/drive/MyDrive/Sarcasm_Data/Xgboost1.model')

In [57]:
y_pred = loaded_bst.predict(deval)
# Convert probabilities to binary output with a 0.5 threshold
y_pred_binary = (y_pred > 0.5).astype(int)


In [58]:
accuracy = accuracy_score(y_eval, y_pred_binary)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6898172194603152


#XGboost Insights
